In [130]:
# -*- coding: utf-8 -*-

import numpy as np


# 1. Lecture des données 

In [99]:
# fonction pour transformer les données brutes en nombres de 0 à n-1
def translate_data ( data ):
# création des structures de données à retourner
    nb_variables = data.shape[0]
    nb_observations = data.shape[1] - 1 # - nom variable
    res_data = np.zeros ( (nb_variables, nb_observations ), int )
    res_dico = np.empty ( nb_variables, dtype=object )
    
    # pour chaque variable, faire la traduction
    for i in range ( nb_variables ):
        res_dico[i] = {}
        index = 0
        for j in range ( 1, nb_observations + 1 ):
            # si l'observation n'existe pas dans le dictionnaire, la rajouter
            if data[i,j] not in res_dico[i]:
                res_dico[i].update ( { data[i,j] : index } )
                index += 1
            # rajouter la traduction dans le tableau de données à retourner
            res_data[i,j-1] = res_dico[i][data[i,j]]
    return ( res_data, res_dico )

In [11]:
# fonction pour lire les données de la base d'apprentissage
def read_csv ( filename ):
    data = np.loadtxt ( filename, delimiter=',', dtype=np.str ).T
    names = data[:,0].copy ()
    data, dico = translate_data ( data )
    return names, data, dico

# names : tableau contenant les noms des variables aléatoires
# data  : tableau 2D contenant les instanciations des variables aléatoires
# dico  : tableau de dictionnaires contenant la correspondance (valeur de variable -> nombre)
names, data, dico = read_csv ( "2015_tme5_asia.csv" )

In [32]:
# etant donné une BD data et son dictionnaire, cette fonction crée le
# tableau de contingence de (x,y) | z
def create_contingency_table ( data, dico, x, y, z ):
# détermination de la taille de z
    size_z = 1
    offset_z = np.zeros ( len ( z ) )
    j = 0
    for i in z:
        offset_z[j] = size_z
        size_z *= len ( dico[i] )
        j += 1
    # création du tableau de contingence
    res = np.zeros ( size_z, dtype = object )
    # remplissage du tableau de contingence
    if size_z != 1:
        z_values = np.apply_along_axis ( lambda val_z : val_z.dot ( offset_z ),1, data[z,:].T )
        i = 0
        while i < size_z:
            indices, = np.where ( z_values == i )
            a,b,c = np.histogram2d ( data[x,indices], data[y,indices],bins = [ len ( dico[x] ), len (dico[y] ) ] )
            res[i] = ( indices.size, a )
            i += 1
    else:
        a,b,c = np.histogram2d ( data[x,:], data[y,:],bins = [ len ( dico[x] ), len (dico[y] ) ] )
        res[0] = ( data.shape[1], a )
    return res

In [100]:
def sufficient_statistics ( data, dico, x, y, z ):
    res = create_contingency_table ( data, dico, x, y, z )
    s=0
    
    for z in range(len(res)):
        Nz = res[z][0]
        if Nz != 0:
            #la variable X est representé sur les lignes
            #pour virer y de Nxz on somme sur les colonnes 
            Nyz = np.sum(res[z][1],axis=0)  #somme colonne
            #la variable y est representé sur les colonnes
            #pour virer x de Nyz on somme sur les lignes
            Nxz =  np.sum(res[z][1],axis=1) #somme ligne 
            
            for i in range(len(res[z][1])):
                for j in range(len(res[z][1])):
                    tmp = (Nxz[i]*Nyz[j]) / Nz # Nxz[i]*Nyz[j] produit terme a terme et non matricielle 
                    if tmp != 0:
                        s = s + ((res[z][1][i][j]-tmp)**2)/tmp
                      
    return s

In [106]:
res = create_contingency_table ( data, dico,1,2,[3])

In [107]:
res

array([(103, array([[  0.,  96.],
       [  0.,   7.]])),
       (1897, array([[ 887.,   18.],
       [ 983.,    9.]]))], dtype=object)

In [101]:
#teste
res = sufficient_statistics ( data, dico, 1,2,[3])
print(res)
res = sufficient_statistics ( data, dico, 0,1,[2,3])
print(res)
res = sufficient_statistics ( data, dico, 1,3,[2])
print(res)
res = sufficient_statistics ( data, dico, 5,2,[1,3,6])
print(res)
res = sufficient_statistics ( data, dico, 0,7,[4,5])
print(res)
res = sufficient_statistics ( data, dico, 2,3,[5])
print(res)

3.94665911867
16.3552074624
81.8074493481
1897.0
3.22232377609
130.0


# 3. Statistique du χ2 et degré de liberté 

In [102]:
 dico

array([{'true': 0, 'false': 1}, {'false': 0, 'true': 1},
       {'true': 0, 'false': 1}, {'false': 0, 'true': 1},
       {'false': 0, 'true': 1}, {'false': 0, 'true': 1},
       {'false': 0, 'true': 1}, {'false': 0, 'true': 1}], dtype=object)

In [126]:
def sufficient_statistics2( data, dico, x, y, z ):
    res = create_contingency_table ( data, dico, x, y, z )
    s=0
    Z=0
    for z in range(len(res)):
        Nz = res[z][0]
        if Nz != 0:
            Z = Z + 1 # valeurs possibles non nulle que peut prendre la variable z
                      
            #la variable X est representé sur les lignes
            #pour virer y de Nxz on somme sur les colonnes 
            Nyz = np.sum(res[z][1],axis=0)  #somme colonne
            #la variable y est representé sur les colonnes
            #pour virer x de Nyz on somme sur les lignes
            Nxz =  np.sum(res[z][1],axis=1) #somme ligne 
            
            for i in range(len(res[z][1])):
                for j in range(len(res[z][1])):
                    tmp = (Nxz[i]*Nyz[j]) / Nz # Nxz[i]*Nyz[j] produit terme a terme et non matricielle 
                    if tmp != 0:
                        s = s + ((res[z][1][i][j]-tmp)**2)/tmp

    X=len(dico[x]) # valeurs possibles que peut prendre la variable x
    Y=len(dico[y]) # valeurs possibles que peut prendre la variable y
    DL = (X-1)*(Y-1)*Z
    return s,DL

In [128]:
#teste
res,dl = sufficient_statistics2 ( data, dico, 1,2,[3])
print(res,dl)
res,dl = sufficient_statistics2( data, dico, 0,1,[2,3])
print(res,dl)
res,dl = sufficient_statistics2 ( data, dico, 1,3,[2])
print(res,dl)
res,dl = sufficient_statistics2 ( data, dico, 5,2,[1,3,6])
print(res,dl)
res,dl = sufficient_statistics2 ( data, dico, 0,7,[4,5])
print(res,dl)
res,dl = sufficient_statistics2 ( data, dico, 2,3,[5])
print(res,dl)

3.94665911867 2
16.3552074624 3
81.8074493481 2
1897.0 8
3.22232377609 4
130.0 2


# 4. Test d'indépendance 

In [139]:
def indep_score(data, dico, x, y, z ):
    x,dl =sufficient_statistics2( data, dico, x, y, z )
    d_min = 5*dl 
    if d_min > len(data[0]): #prérequis obligatoire 5*|X|*|Y|*|Z| < Nbr ligne de notre base 
        return -1,1
    return (stats.chi2.sf(x,dl)) 
# Regle : 
# si chi2_observé > seuil de risque (alpha) alors x et y sont indep conditionnellement a z
# sinon x et y sont depedants


In [142]:
#teste
indep = indep_score ( data, dico, 1,3,[])
print(indep)
indep = indep = indep_score ( data, dico, 1, 7, [])
print(indep)
indep = indep_score ( data, dico, 0, 1,[2, 3])
print(indep)
indep = indep_score ( data, dico, 1, 2,[3, 4])
print(indep)

2.38520176938e-19
1.12562784979e-10
0.000958828236575
0.475266197894
